# Load Store Handler Class

In [14]:
from linkingtool.hdf5_handler import DataHandler
import linkingtool.visuals as vis

# Define Province Code

In [44]:
province_code:str='AB'

In [ ]:
store=f"data/store/resources_{province_code}.h5"
datahandler=DataHandler(store)
datahandler.show()

In [ ]:
timeseries_clusters_solar=datahandler.from_store('timeseries_clusters_solar')
timeseries_clusters_wind=datahandler.from_store('timeseries_clusters_wind')
cells=datahandler.from_store('cells')
clusters_solar=datahandler.from_store('clusters_solar')

In [ ]:
cells.dissolve(by='Province')

In [50]:
import pandas as pd
import matplotlib.pyplot as plt

# Assuming 'cells' is your GeoDataFrame or DataFrame with a 'Region' column
# cells.plot('Region', edgecolor='yellow', linewidth=0.2)

# Create the plot
ax = cells.dissolve(by='Region').plot(edgecolor='none',color='skyblue',alpha=0.5)

# Turn off the grid
ax.grid(False)

# Save the figure as an SVG file
plt.savefig('plot2.svg', format='svg')

# Optionally, you can also close the plot if you don't want to display it
plt.close()


In [51]:
import pandas as pd
import matplotlib.pyplot as plt

# Assuming 'cells' is your GeoDataFrame or DataFrame with a 'Region' column
# cells.plot('Region', edgecolor='yellow', linewidth=0.2)

# Create the plot
ax = cells.plot('Region', edgecolor='yellow', linewidth=0.2)

# Turn off the grid
ax.grid(False)

# Save the figure as an SVG file
plt.savefig('plot.svg', format='svg')

# Optionally, you can also close the plot if you don't want to display it
plt.close()


In [ ]:
import pandas as pd
import plotly.graph_objects as go

# Assuming 'df' is your DataFrame containing time series data
df = timeseries_clusters_solar  # Load your DataFrame

# Define a list of colors (you can customize this list as needed)
colors = ['blue', 'orange', 'green', 'red', 'purple', 'brown', 'pink', 'gray']  # Add more colors if needed

# Create a list of traces, one for each column
traces = []
for i, col in enumerate(df.columns):
    trace = go.Scatter(
        x=df.index,
        y=df[col],
        mode='lines',
        name=col,
        visible=False,
        line=dict(color=colors[i % len(colors)])  # Cycle through colors
    )
    traces.append(trace)

# Make the first column visible initially
traces[0]['visible'] = True

# Create dropdown options for each column
dropdown_buttons = []
for i, col in enumerate(df.columns):
    button = dict(
        args=[{'visible': [False] * len(df.columns)}],  # Make all traces invisible first
        label=col,
        method='update'
    )
    button['args'][0]['visible'][i] = True  # Make only the selected trace visible
    dropdown_buttons.append(button)

# Define the layout for the figure
layout = go.Layout(
    title="Time Series - Solar",
    xaxis_title="DateTime",
    yaxis_title="Value",
    updatemenus=[{
        'buttons': dropdown_buttons,
        'direction': 'down',
        'showactive': True
    }]
)

# Create the figure with all traces and layout
fig = go.Figure(data=traces, layout=layout)

# Show the figure
fig.show()


In [ ]:
import pandas as pd
import plotly.graph_objects as go

# Assuming 'df' is your DataFrame containing time series data
df = timeseries_clusters_wind  # Load your DataFrame

# Define a list of colors (you can customize this list as needed)
colors = ['blue', 'orange', 'green', 'red', 'purple', 'brown', 'pink', 'gray']  # Add more colors if needed

# Create a list of traces, one for each column
traces = []
for i, col in enumerate(df.columns):
    trace = go.Scatter(
        x=df.index,
        y=df[col],
        mode='lines',
        name=col,
        visible=False,
        line=dict(color=colors[i % len(colors)])  # Cycle through colors
    )
    traces.append(trace)

# Make the first column visible initially
traces[0]['visible'] = True

# Create dropdown options for each column
dropdown_buttons = []
for i, col in enumerate(df.columns):
    button = dict(
        args=[{'visible': [False] * len(df.columns)}],  # Make all traces invisible first
        label=col,
        method='update'
    )
    button['args'][0]['visible'][i] = True  # Make only the selected trace visible
    dropdown_buttons.append(button)

# Define the layout for the figure
layout = go.Layout(
    title="Time Series- Wind",
    xaxis_title="DateTime",
    yaxis_title="Value",
    updatemenus=[{
        'buttons': dropdown_buttons,
        'direction': 'down',
        'showactive': True
    }]
)

# Create the figure with all traces and layout
fig = go.Figure(data=traces, layout=layout)

# Show the figure
fig.show()


In [ ]:
cells

In [ ]:
# data1=solar.get_CF_timeseries()
# data2=solar.get_cell_capacity()
# data3=solar.find_grid_nodes()
# data5=solar.score_cells()
# data=datahandler.from_store('cells') 
vis.get_selected_vs_missed_visuals(cells, 
                              'BC',
                              'solar',
                              lcoe_threshold=20, 
                              CF_threshold=0.15, 
                              capacity_threshold=1,
                              text_box_x=.65,
                              text_box_y=.965,
                              title_x=.65,
                              title_y=1,
                              font_size=8,
                              dpi=400,
                              figsize=(8,6),
                              save=True
                              )

In [8]:
lcoe_threshold=20
wcss_tolr=0.05
filtered_cells=cells[cells['lcoe_solar'] <= lcoe_threshold]
filtered_clusters=clusters_solar[clusters_solar['lcoe_solar'] <= lcoe_threshold]

In [ ]:
import geopandas as gpd
import matplotlib.pyplot as plt

# Define the LCOE threshold
lcoe_threshold = 20  # You can set this to the appropriate value as needed

# Create a figure with 1 row and 2 columns
fig, ax = plt.subplots(1, 2, figsize=(12, 6))

# Plot the selected cells on the left
filtered_cells.plot(
    column='lcoe_solar',
    ax=ax[0],
    edgecolor='white',
    linewidth=0.2,
    legend=True,
    legend_kwds={'label': "Cells LCOE ($/kWh)", 'orientation': "vertical"}
)
ax[0].set_title('Cells')
ax[0].set_xlabel('Longitude')
ax[0].set_ylabel('Latitude')

# Plot the filtered clusters on the right
filtered_clusters.plot(
    column='lcoe_solar',
    ax=ax[1],
    edgecolor='white',
    linewidth=0.2,
    legend=True,
    # legend_kwds={'label': "Cluster LCOE ($/kWh)", 'orientation': "vertical"}
)
ax[1].set_title(f'Representative Clusters [@ WCSS tolerance {wcss_tolr}')
ax[1].set_xlabel('Longitude')
ax[1].set_ylabel('Latitude')

# Set a common title for the entire figure
fig.suptitle(f"BC- Selected Cells vs their Clusters \n[@ lcoe >={lcoe_threshold} $/kWh]", fontsize=16, fontweight='bold')

# Adjust layout
plt.tight_layout()
plt.subplots_adjust(top=0.85)  # Adjusts the top space to make room for the main title
plt.savefig(f'vis/Solar/cells vs cluster_ALL REGIONS_BC')
# Show the plots
plt.show()
